# <font color=green>1. MEU PRIMEIRO SCRAPING

## 1. Ambiente e bibliotecas

In [1]:
import time
import re
import math

import bs4
import urllib.request as urllib_request
import pandas as pd

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.11.1
urllib -> 3.8
pandas -> 1.5.2


In [2]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

## 2. Data and Dictionary

In [3]:
def get_keys():
    data = {'Picture Gallery/Hope Board': 0, 'Camera': 1, 'Personality Disorders': 2, 'References/Information': 3, 'App Reports Security Measures in Place': 4, 'Collaborative With Provider/Other': 5, 'Track Mood': 6, 'Geolocation': 7, 'Hybrid': 8, 'Subscription': 9, 'Accessibility': 10, 'Well Written Relevant Content': 11, 'Data Stored on Device': 12, 'Substance Use': 13, 'iCBT or Sleep Therapy': 14, 'For Profit Company': 15, 'DBT': 16, 'Coach/Therapist Connection': 17, 'Eating Disorders': 18, 'Pain': 19, 'Self-Harm': 20, 'Spanish': 21, 'Patient Facing': 22, 'Can Delete Data': 23, 'Sleep': 24, 'Gamification (Points/Badges)': 25, 'Email or Export Your Data': 26, 'Track Symptoms': 27, 'Healthcare Company': 28, 'Graphs of Data': 29, 'Academic Institution': 30, 'Real Time Response': 31, 'User Generated Data': 32, 'Does What it Claims': 33, 'Payment': 34, 'Deep Breathing': 35, 'External Devices': 36, 'Is PHI Shared': 37, 'Claims to Not Replace Care': 38, 'Notifications': 39, 'Reminders': 40, 'Meets HIPAA': 41, 'CBT': 42, 'Send Your Data to a Medical Record': 43, 'Assessments/Screenings': 44, 'Has Crisis Management Feature': 45, 'Peer Support': 46, 'Self Help': 47, 'App Declares Data Use and Purpose': 48, 'AI Support': 49, 'In-App Purchase': 50, 'Summary of Data': 51, 'Psychoeducation': 52, 'Social Network': 53, 'Substance Use (Alcohol)': 54, 'Mood Disorders': 55, 'Reference': 56, 'Substance Use (Smoking & Tobacco)': 57, 'Free to Download': 58, 'Mindfulness': 59, 'Non-Profit Company': 60, 'Supporting Studies': 61, 'Data Stored on Server': 62, 'Offline': 63, 'PTSD': 64, 'Schizophrenia': 65, 'Videos': 66, 'OCD': 67, 'Physical Health Exercises': 68, 'Productivity': 69, 'Biodata': 70, 'Link to Formal Care/Coaching': 71, 'Phobias': 72, 'De-Identified/Anonymized Data Shared': 73, 'Government': 74, 'Headache': 75, 'Diary': 76, 'Track Medication': 77, 'Chat/Message': 78, 'Track Sleep': 79, 'Microphone': 80, 'Network Support': 81, 'Contact List': 82, 'Bipolar Disorder': 83, 'Stress & Anxiety': 84, 'Surveys': 85, 'Goal Setting/Habits': 86, 'Bio Feedback with Sense Data': 87, 'ACT': 88, 'Can Cause Harm': 89, 'Aggregated Data Shared': 90, 'Has Privacy Policy': 91, 'Chatbot Interaction': 92, 'Can Opt Out of Data Collection': 93, 'Physical Health': 94, 'Totally Free': 95, 'Journaling': 96, 'Asynchronous Response': 97, 'Own Your Own Data': 98, 'Audio/Music/Scripts': 99, 'Non-Specific': 100, 'Appropriately Advises Patient in Case of Emergency': 101}
    return data

In [4]:
def get_dataframe(data):
    keys = ['Name'] + [key for key in data]
    df = pd.DataFrame(columns=keys)
    
    return df

In [5]:
cost_opt = set()
privacy_opt = set()
foundation_opt = set()
features_opt = set()
conditions_opt = set()
engagements_opt = set()
inputs_opt = set()
outputs_opt = set()
uses_opt = set()
dev_types_opt = set()
access_opt = set()

## 3. Functions

In [6]:
def str_to_valid_str(str_data):
    return " ".join(str_data.split()).replace("> <", "><")

In [7]:
def bytes_to_valid_str(b_data):
    str_data = b_data.decode("utf-8")
    return str_to_valid_str(str_data)

In [8]:
def get_html(page):
    html = str_to_valid_str(page)
    
    soup = BeautifulSoup(html, "html.parser")
    return soup.prettify()

In [9]:
def click_button(driver, btn):
    driver.execute_script("arguments[0].click();", btn)

In [10]:
def close_popup(driver):
    try:
        close_btn = None
        btns = driver.find_elements(By.TAG_NAME, 'button')
        for btn in btns:
            if btn.text == "Close":
                print("    Found button")
                close_btn = btn
        if close_btn != None:
            click_button(driver, close_btn)

    except StaleElementReferenceException as e:
        print("\n\n\n    Popup closed ", e.reason)

In [11]:
def change_to_table_view(driver):
    try:
        close_btn = None
        btns = driver.find_elements(By.TAG_NAME, 'button')
        for btn in btns:
            if btn.text == "Table View":
                close_btn = btn
        click_button(driver, close_btn)

    except StaleElementReferenceException as e:
        print("\n\n\n    Change to table view ", e.reason)

In [12]:
def scroll_page(driver, element, value):
    driver.execute_script("arguments[0].scroll(0," + str(value) + ");", element)

In [13]:
def get_from_xpath(driver, path):
    return driver.find_element(By.XPATH, path)

In [14]:
def get_data(driver, df, data):
    column = list(df.columns.values)
    time.sleep(5) # Load page
    # Get info about number of comments
    coments_info = driver.find_element(
        By.XPATH, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[4]'
    )

    time.sleep(10) # Load info
    quant = int(re.split('\)|\(', coments_info.text)[1]) # Get coments quantity
    
    # Get header data
    name = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[2]/div/div[2]/div/div/div/div[1]/h5')
    supported_study = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[2]/div/div[3]/div/div[3]/div/div[2]/div/div[2]/span')
    
    coments_quant[name.text] = quant
    print("name: ", name.text, " has study: ", supported_study.text, " comentários: ", quant)
    # Get each comment    
    for i in range(quant):
        # Get data
        cost = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[3]')
        privacy = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[4]')
        foundation = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[5]')
        features = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[6]')
        conditions = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[7]')
        engagements = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[8]')
        inputs = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[9]')
        outputs = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[10]')
        uses = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[11]')
        dev_types = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[12]')
        access = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[13]')
    
        # Get tags
        tags = re.split('\n', cost.text)[1:]
        tags = tags + re.split('\n', privacy.text)[1:]
        tags = tags + re.split('\n', foundation.text)[1:]
        tags = tags + re.split('\n', features.text)[1:]
        tags = tags + re.split('\n', conditions.text)[1:]
        tags = tags + re.split('\n', engagements.text)[1:]
        tags = tags + re.split('\n', inputs.text)[1:]
        tags = tags + re.split('\n', outputs.text)[1:]
        tags = tags + re.split('\n', uses.text)[1:]
        tags = tags + re.split('\n', dev_types.text)[1:]
        tags = tags + re.split('\n', access.text)[1:]
        
        # Assemble line
        base_line = [name.text] + ([0] * (len(column) - 1))
        print("base_line X dictionary: ", len(base_line), " = ", len(column))
        for index in tags:
            base_line[data[index] + 1] = 1

        # Add line to dataframe
        new_line = pd.DataFrame([base_line], columns=column)
        df = pd.concat([df, new_line], ignore_index = True)
        
        # Update tags database
        for item in re.split('\n', cost.text)[1:]:
            cost_opt.add(item)
        for item in re.split('\n', privacy.text)[1:]:
            privacy_opt.add(item)
        for item in re.split('\n', foundation.text)[1:]:
            foundation_opt.add(item)
        for item in re.split('\n', features.text)[1:]:
            features_opt.add(item)
        for item in re.split('\n', conditions.text)[1:]:
            conditions_opt.add(item)
        for item in re.split('\n', engagements.text)[1:]:
            engagements_opt.add(item)
        for item in re.split('\n', inputs.text)[1:]:
            inputs_opt.add(item)
        for item in re.split('\n', outputs.text)[1:]:
            outputs_opt.add(item)
        for item in re.split('\n', uses.text)[1:]:
            uses_opt.add(item)
        for item in re.split('\n', dev_types.text)[1:]:
            dev_types_opt.add(item)
        for item in re.split('\n', access.text)[1:]:
            access_opt.add(item)
        
        # Button to the next comment
        if i < quant - 1:
            next_btn = driver.find_element(By.CSS_SELECTOR,
                '#app-content > div.jss5.jss8 > div > div:nth-child(4) > div > div:nth-child(6) > div > nav > ul > li:last-child > button'
            )
            click_button(driver, next_btn) # Go to comment
            time.sleep(10) # Load info
    
    return df

## 4. Web scrapping com Selenium 

In [15]:
def selenium_scrapping(start, end):
    options = Options()
    # options.headless = True
    options.add_argument("--window-size=1920,820")

    dictionary = get_keys()
    df = get_dataframe(dictionary)

    DRIVER_PATH = '/home/pizzaisamor/chromedriver/chromedriver'
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

    driver.get('https://mindapps.org/Apps')
    driver.set_window_position(150, 150)
    driver.set_window_size(1920, 820)

    try:
        element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "app-content"))
        )
    
        # close_popup(driver)
        time.sleep(15) # Load page
    
        for card in range(start, end):
            print("\nCard: ", card)
        
            # Scroll apps list
            apps_list = driver.find_element(
                By.XPATH, '//*[@id="app-content"]/div[2]/div[3]/div[3]/div/div/div[1]'
            )
            line = math.floor(card / 4) # Find right line 
            scroll_page(driver, apps_list, line * 536) # Go to the line
            time.sleep(3) # Wait to update DOM
        
            # Find card
            navigate = driver.find_element(
                By.XPATH, '//div[@data-index='+ str(card) +']/div'
            )
    
            click_button(driver, navigate) # Click on the card
            df = get_data(driver, df, dictionary) # Get the data from the page
            driver.back() # Go to main menu
            time.sleep(8) # Load page

        print("\n\nDict:")
        for key in coments_quant.keys():
            print(key, " = ", coments_quant[key])
        
        display(df)
    
    finally:
        driver.quit()
        return df

In [16]:
# Create categories dictionary
categories = set()

for index in cost_opt:
    categories.add(index)
for index in privacy_opt:
    categories.add(index)
for index in foundation_opt:
    categories.add(index)
for index in features_opt:
    categories.add(index)
for index in conditions_opt:
    categories.add(index)
for index in engagements_opt:
    categories.add(index)
for index in inputs_opt:
    categories.add(index)
for index in outputs_opt:
    categories.add(index)
for index in uses_opt:
    categories.add(index)
for index in dev_types_opt:
    categories.add(index)
for index in access_opt:
    categories.add(index)
    
count = 0
dictionary = dict()
for index in categories:
    dictionary[index] = count
    count += 1

print(dictionary)

{}


In [17]:
coments_quant = dict()
dataset = get_dataframe(get_keys())
count = 0
seq_size = 20

# Start in 0 end in 612
for seq in range(0, 612, seq_size):
    print("Intervalo: ", seq, " - ", (seq + seq_size))
    df = selenium_scrapping(seq, seq + seq_size)
    
    dataset = pd.concat([dataset, df], ignore_index = True)
    
    file_name = 'web-scrapping_v' + str(count) + '.csv'
    dataset.to_csv(file_name, encoding='utf-8', index=False)
    count += 1

Intervalo:  0  -  20


/tmp/ipykernel_27691/2726943785.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)



Card:  0
name:  Intellect: Create a Better You  has study:  No  comentários:  1
base_line X dictionary:  103  =  103

Card:  1
name:  Bipolar Test  has study:  Yes  comentários:  8
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103

Card:  2
name:  Pain Rating Scales  has study:  No  comentários:  4
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103

Card:  3
name:  CBT Guide to Depression Self-help: Mood Log, Diary  has study:  No  comentários:  7
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103
base_line X diction

,Name,Picture Gallery/Hope Board,Camera,Personality Disorders,References/Information,App Reports Security Measures in Place,Collaborative With Provider/Other,Track Mood,Geolocation,Hybrid,...,Chatbot Interaction,Can Opt Out of Data Collection,Physical Health,Totally Free,Journaling,Asynchronous Response,Own Your Own Data,Audio/Music/Scripts,Non-Specific,Appropriately Advises Patient in Case of Emergency
0,Intellect: Create a Better You,0,0,0,1,1,0,1,0,0,...,0,0,0,0,1,0,1,1,1,0
1,Bipolar Test,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
2,Bipolar Test,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
3,Bipolar Test,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
4,Bipolar Test,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Migraine Coach,0,0,0,0,1,0,0,1,1,...,1,0,0,1,1,0,1,0,0,0
97,Migraine Coach,0,0,0,0,1,0,0,1,1,...,1,0,0,1,1,0,0,0,0,0
98,Migraine Coach,0,0,0,0,1,0,0,1,1,...,1,0,0,1,1,0,0,0,0,0
99,Migraine Coach,0,0,0,0,1,0,0,1,1,...,1,0,0,1,1,0,0,0,0,0


Intervalo:  20  -  40


...


Card:  20
name:  MEDITATION BY MT – SELF HELP, STRESS RELIEF  has study:  No  comentários:  1
base_line X dictionary:  103  =  103

Card:  21
name:  TalkLife for Anxiety, Depression & Stress  has study:  No  comentários:  1
base_line X dictionary:  103  =  103

Card:  22
name:  Chronic Pain Tracker Lite  has study:  No  comentários:  1
base_line X dictionary:  103  =  103

Card:  23
name:  TF-CBT Triangle of Life  has study:  No  comentários:  2
base_line X dictionary:  103  =  103
base_line X dictionary:  103  =  103

Card:  24
name:  Joyable: An AbleTo Program  has study:  No  comentários:  1
base_line X dictionary:  103  =  103

Card:  25
name:  CBT-i Coach  has study:  No  comentários:  1
base_line X dictionary:  103  =  103

Card:  26
name:  UCLA Mindful  has study:  No  comentários:  1
base_line X dictionary:  103  =  103

Card:  27
name:  Deep Sleep, Insomnia Help  has study:  No  comentários:  1
base_line X dictionary:  103  =  103

Card:  28
name:  minding - meditation timer 

,Name,Picture Gallery/Hope Board,Camera,Personality Disorders,References/Information,App Reports Security Measures in Place,Collaborative With Provider/Other,Track Mood,Geolocation,Hybrid,...,Chatbot Interaction,Can Opt Out of Data Collection,Physical Health,Totally Free,Journaling,Asynchronous Response,Own Your Own Data,Audio/Music/Scripts,Non-Specific,Appropriately Advises Patient in Case of Emergency
0,"MEDITATION BY MT – SELF HELP, STRESS RELIEF",1,1,0,0,1,0,1,1,0,...,0,1,1,0,1,0,1,1,0,0
1,"TalkLife for Anxiety, Depression & Stress",0,1,0,0,1,0,1,1,0,...,0,0,0,0,1,1,0,0,1,1
2,Chronic Pain Tracker Lite,0,0,0,0,0,0,1,1,0,...,0,0,1,0,0,0,1,0,0,0
3,TF-CBT Triangle of Life,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,1,1,0
4,TF-CBT Triangle of Life,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,Joyable: An AbleTo Program,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
6,CBT-i Coach,0,0,0,1,1,1,0,0,1,...,0,1,0,1,0,0,1,0,0,0
7,UCLA Mindful,0,0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,1,0,0
8,"Deep Sleep, Insomnia Help",0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
9,minding - meditation timer,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0


Intervalo:  40  -  60


/tmp/ipykernel_27691/2726943785.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


WebDriverException: Message: unknown error: net::ERR_ADDRESS_UNREACHABLE
  (Session info: chrome=92.0.4515.131)
Stacktrace:
#0 0x559f6fb55a63 <unknown>
#1 0x559f6f8ca458 <unknown>
#2 0x559f6f8c486d <unknown>
#3 0x559f6f8b68a6 <unknown>
#4 0x559f6f8b77c3 <unknown>
#5 0x559f6f8b6b72 <unknown>
#6 0x559f6f8b6154 <unknown>
#7 0x559f6f8b4f23 <unknown>
#8 0x559f6f8b5273 <unknown>
#9 0x559f6f8cbe1a <unknown>
#10 0x559f6f930a9f <unknown>
#11 0x559f6f91d0e2 <unknown>
#12 0x559f6f93023c <unknown>
#13 0x559f6f91cfd3 <unknown>
#14 0x559f6f8f3514 <unknown>
#15 0x559f6f8f4505 <unknown>
#16 0x559f6fb81e2e <unknown>
#17 0x559f6fb97886 <unknown>
#18 0x559f6fb82d75 <unknown>
#19 0x559f6fb98d94 <unknown>
#20 0x559f6fb798eb <unknown>
#21 0x559f6fbb3cd8 <unknown>
#22 0x559f6fbb3e58 <unknown>
#23 0x559f6fbcddfd <unknown>
#24 0x7fbb04a54609 start_thread
